In [4]:
import fnmatch
import json
import logging
import os
import shutil
import threading
import datetime
import errno
import sys
import time
import re
import zipfile
import tarfile

try:
    from queue import Queue
except ImportError:
    from Queue import Queue

try:
    from urllib.request import urlopen
    from urllib.error import URLError, HTTPError
except ImportError:
    from urllib2 import urlopen, URLError, HTTPError

import boto.dynamodb2.layer1
from boto.dynamodb2.exceptions import ProvisionedThroughputExceededException
import botocore
import boto3

In [12]:
import boto3

# For a Boto3 client.
ddb = boto3.client('dynamodb', endpoint_url='http://localhost:8000')
# response = ddb.list_tables()
# print(response)

In [13]:
ddb.delete_table

SyntaxError: invalid syntax (<ipython-input-13-a857392b301d>, line 1)

In [6]:
class Object(object):
    pass

args = Object()
args.dumpPath = 'dump'
args.dataOnly = False
SCHEMA_FILE = "schema.json"

table = 'asx_raw_prices_2'
dynamo = ddb
sleep_interval = 1
source_table = table
destination_table = table
write_capacity = 100

# if os.path.exists("%s/%s" % (args.dumpPath, source_table)):
dump_data_path = args.dumpPath

In [9]:
os.listdir()

['dump',
 'dynamodump.py',
 'query_test.ipynb',
 '.ipynb_checkpoints',
 '.git',
 '.gitignore']

In [7]:
dump_data_path + os.sep + source_table + os.sep + SCHEMA_FILE

'dump/asx_raw_prices_2/schema.json'

In [8]:
table_data = json.load(open(dump_data_path + os.sep + source_table + os.sep + SCHEMA_FILE))

table = table_data["Table"]
table_attribute_definitions = table["AttributeDefinitions"]
table_table_name = destination_table
table_key_schema = table["KeySchema"]
original_read_capacity = table["ProvisionedThroughput"]["ReadCapacityUnits"]
original_write_capacity = table["ProvisionedThroughput"]["WriteCapacityUnits"]
table_local_secondary_indexes = table.get("LocalSecondaryIndexes")
table_global_secondary_indexes = table.get("GlobalSecondaryIndexes")

FileNotFoundError: [Errno 2] No such file or directory: 'dump/asx_raw_prices_2/schema.json'

In [7]:
table

{'AttributeDefinitions': [{'AttributeName': 'date', 'AttributeType': 'S'}],
 'TableName': 'asx_listed_companies',
 'KeySchema': [{'AttributeName': 'date', 'KeyType': 'HASH'}],
 'TableStatus': 'ACTIVE',
 'CreationDateTime': 1598505676.426,
 'ProvisionedThroughput': {'LastIncreaseDateTime': 0.0,
  'LastDecreaseDateTime': 0.0,
  'NumberOfDecreasesToday': 0,
  'ReadCapacityUnits': 1,
  'WriteCapacityUnits': 1},
 'TableSizeBytes': 5785277,
 'ItemCount': 27,
 'TableArn': 'arn:aws:dynamodb:ddblocal:000000000000:table/asx_listed_companies'}

In [61]:
table_provisioned_throughput = {"ReadCapacityUnits": int(original_read_capacity),
                                    "WriteCapacityUnits": int(write_capacity)}
table_provisioned_throughput

{'ReadCapacityUnits': 1, 'WriteCapacityUnits': 100}

In [62]:
not args.dataOnly

True

In [79]:
# create the table
dynamo.create_table(AttributeDefinitions = table_attribute_definitions,
                    TableName=table_table_name,
                    KeySchema=table_key_schema,
                    ProvisionedThroughput=table_provisioned_throughput)

ResourceInUseException: An error occurred (ResourceInUseException) when calling the CreateTable operation: Cannot create preexisting table

In [85]:
# put data on the table

def batch_write(conn, sleep_interval, table_name, put_requests):
    """
    Write data to table_name
    """

    request_items = {table_name: put_requests}
    i = 1
    sleep = sleep_interval
    while True:
        response = conn.batch_write_item(RequestItems=request_items)
        unprocessed_items = response["UnprocessedItems"]

        if len(unprocessed_items) == 0:
            break
        if len(unprocessed_items) > 0 and i <= MAX_RETRY:
            logging.debug(str(len(unprocessed_items)) +
                          " unprocessed items, retrying after %s seconds.. [%s/%s]"
                          % (str(sleep), str(i), str(MAX_RETRY)))
            request_items = unprocessed_items
            time.sleep(sleep)
            sleep += sleep_interval
            i += 1
        else:
            logging.info("Max retries reached, failed to processed batch write: " +
                         json.dumps(unprocessed_items, indent=JSON_INDENT))
            logging.info("Ignoring and continuing..")
            break
            
            
DATA_DIR = "data"
MAX_BATCH_WRITE = 25  # DynamoDB limit
BATCH_WRITE_SLEEP_INTERVAL = 0.15  # seconds

logging.info("Restoring data for " + destination_table + " table..")
data_file_list = os.listdir(dump_data_path + os.sep + source_table +
                            os.sep + DATA_DIR + os.sep)
data_file_list.sort()

for data_file in data_file_list:
    logging.info("Processing " + data_file + " of " + destination_table)
    items = []
    item_data = json.load(
        open(
            dump_data_path + os.sep + source_table + os.sep + DATA_DIR + os.sep + data_file
        )
    )
    items.extend(item_data["Items"])

    # batch write data
    put_requests = []
    while len(items) > 0:
        put_requests.append({"PutRequest": {"Item": items.pop(0)}})

        # flush every MAX_BATCH_WRITE
        if len(put_requests) == MAX_BATCH_WRITE:
            logging.debug("Writing next " + str(MAX_BATCH_WRITE) +
                          " items to " + destination_table + "..")
            batch_write(dynamo, BATCH_WRITE_SLEEP_INTERVAL, destination_table, put_requests)
            del put_requests[:]

    # flush remainder
    if len(put_requests) > 0:
        batch_write(dynamo, BATCH_WRITE_SLEEP_INTERVAL, destination_table, put_requests)



In [15]:
ddb = boto3.client('dynamodb', endpoint_url='http://localhost:8000')
response = ddb.list_tables()
print(response)

{'TableNames': ['asx_listed_companies', 'asx_raw_prices_2'], 'ResponseMetadata': {'RequestId': 'f2eb923e-3608-43e2-bab6-a00191109077', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0', 'x-amz-crc32': '3974150834', 'x-amzn-requestid': 'f2eb923e-3608-43e2-bab6-a00191109077', 'content-length': '58', 'server': 'Jetty(8.1.12.v20130726)'}, 'RetryAttempts': 0}}


In [36]:
ddb = boto3.resource('dynamodb', endpoint_url='http://localhost:8000')
from boto3.dynamodb.conditions import Key, Attr
date = '2020-01-07'
table = 'asx_raw_prices_2'
table_name = ddb.Table(table)
table_name.query(KeyConditionExpression=Key('date').eq(date))

ValueError: Invalid endpoint: ap-southeast-2

In [34]:
table_scan = table_name.scan()
len(table_scan['Items'])

7

In [35]:
for item in table_scan['Items']:
    print(item['date'])

2014-06-04
2017-06-06
2015-07-28
2014-01-23
2016-11-10
2018-02-02
2018-09-06


In [35]:
table = 'asx_listed_companies'
do_restore(dynamo = ddb, sleep_interval = 1, source_table = table,
           destination_table = table, write_capacity = 100)

TypeError: create_table() only accepts keyword arguments.